In [3]:
class Empty(Exception):
    pass
class Full(Exception):
    pass


In [4]:
class Queue:
    def enqueue(self, item):
        pass
    def dequeue(self):
        pass
    def is_empty(self):
        return True
    def first(self):
        pass
    def __len__(self):
        pass
    def _check_empty(self):
        if self.is_empty():
            raise Empty("Queue is empty")


In [5]:
class ArrayQueue1(Queue):
    """simple array queue"""
    def __init__(self):
        self._items = []
    def __len__(self):
        return len(self._items)
    def is_empty(self):
        print('is empty of aq1')
        return len(self) == 0
    def _check_empty(self):
        if self.is_empty():
            raise Empty("Queue is empty")
    def enqueue(self, item):
        self._items.append(item)
    def dequeue(self):
        self._check_empty()
        return self._items.pop(0)
    def first(self):
        self._check_empty()
        return self._items[0]
    

def test_array_queue_1(items):
    queue = ArrayQueue1()
    for item in items:
        queue.enqueue(item)
    if len(items) > 0:
        queue.first() == items[0]
        assert queue.is_empty() == False
    for item in items:
        assert item == queue.dequeue()
    assert queue.is_empty() == True
    

test_array_queue_1([1, 2, 3, 4, 5])
test_array_queue_1([])
    

is empty of aq1
is empty of aq1
is empty of aq1
is empty of aq1
is empty of aq1
is empty of aq1
is empty of aq1
is empty of aq1
is empty of aq1


In [6]:
class ArrayQueue2(ArrayQueue1):
    """drifting array queue"""
    def __init__(self):
        super().__init__()
        self._first = 0
    def __len__(self):
        length = len(self._items)
        length = length - self._first
        return length
    def _incr_first(self):
        self._first+=1
    
    def dequeue(self):
        self._check_empty()
        item = self._items[self._first]
        self._items[self._first] = None
        self._incr_first()
        return item
    def first(self):
        self._check_empty()
        return self._items[self._first]
    def is_empty(self):
        return len(self) == 0

    def __str__(self):
        return str(self._items)
    def __repr__(self):
        return str(self._items)
        
        

def test_array_queue_2(items):
    queue = ArrayQueue2()
    for item in items:
        queue.enqueue(item)
    if not queue.is_empty():
        queue.first() == items[0]
        assert queue.is_empty() == False
    for item in items:
        # print(queue)
        assert queue.dequeue() == item

    assert queue.is_empty() == True
    for item in items:
        queue.enqueue(item)
    if not queue.is_empty():
        queue.first() == items[0]
        assert queue.is_empty() == False
    for item in items:
        # print(queue)
        assert queue.dequeue() == item
    print(queue)
    

test_array_queue_2([1, 2, 3, 4, 5])
test_array_queue_2([])
    


[None, None, None, None, None, None, None, None, None, None]
[]


In [7]:
class ArrayQueue3(Queue):
    """circular array non resizing queue"""
    def __init__(self, capacity):
        self._items = [None] * capacity
        self._front = 0
        self._capacity = capacity
        self._size = 0
    def __size_pp(self):
        self._size+=1
    def __size_mm(self):
        self._size-=1
    def __len__(self):
        return self._size
    def is_empty(self):
        return len(self) == 0
    def _check_empty(self):
        if self.is_empty():
            raise Empty("Queue is empty")
    def first(self):
        self._check_empty()
        return self._items[self._front]
    def dequeue(self):
        self._check_empty()
        item = self._items[self._front]
        self._items[self._front] = None
        self._front = (self._front + 1) % self._capacity
        self.__size_mm()
        return item
    def enqueue(self, item):
        if self._size == self._capacity:
            raise Full("Queue is full")
        avail = (self._front + self._size) % self._capacity
        self._items[avail] = item
        self.__size_pp()
    def __str__(self):
        return str(self._items)
        

def test_array_queue_3(items):
    queue = ArrayQueue3(len(items))
    for item in items:
        queue.enqueue(item)
    if not queue.is_empty():
        queue.first() == items[0]
        assert queue.is_empty() == False
    for item in items:
        print(queue)
        assert queue.dequeue() == item

    assert queue.is_empty() == True
    for item in items:
        queue.enqueue(item)
    try:
        for item in items:
            queue.enqueue(item)
    except Full as full:
        print(full)
    if not queue.is_empty():
        queue.first() == items[0]
        assert queue.is_empty() == False
    for item in items:
        print(queue)
        assert queue.dequeue() == item
    print(queue)
    

test_array_queue_3([1, 2, 3, 4, 5])
test_array_queue_3([])

[1, 2, 3, 4, 5]
[None, 2, 3, 4, 5]
[None, None, 3, 4, 5]
[None, None, None, 4, 5]
[None, None, None, None, 5]
Queue is full
[1, 2, 3, 4, 5]
[None, 2, 3, 4, 5]
[None, None, 3, 4, 5]
[None, None, None, 4, 5]
[None, None, None, None, 5]
[None, None, None, None, None]
[]


In [11]:
class ArrayQueue4(ArrayQueue3):
    """circular resizing array queue"""
    MINIMUM_CAPACITY = 8
    def __size_pp(self):
        self._size+=1
    def __size_mm(self):
        self._size-=1
    def enqueue(self, item):
        if self._size == self._capacity:
            self._resize(2 * self._capacity)
        avail = (self._front + self._size) % self._capacity
        self._items[avail] = item
        self.__size_pp()
    def dequeue(self):
        # print(self._front, self._size, self._capacity)
        self._check_empty()
        item = self._items[self._front]
        self._items[self._front] = None
        self._front = (self._front + 1) % self._capacity
        self.__size_mm()
        if 0 < self._size and self._size < self._capacity // 4 and self._capacity // 2 > ArrayQueue4.MINIMUM_CAPACITY:
            self._resize(self._capacity // 2)
        return item
    def _resize(self, capacity):
        old = self._items
        new = [None] * capacity
        walk = self._front
        for k in range(self._size):
            new[k] = old[walk]
            walk = (1 + walk) % self._capacity
        self._front = 0
        self._capacity = capacity
        self._items = new
    
    
def test_array_queue_4(items):
    queue = ArrayQueue4(len(items))
    for item in items:
        queue.enqueue(item)
    if not queue.is_empty():
        queue.first() == items[0]
        assert queue.is_empty() == False
    for item in items:
        print(queue)
        assert queue.dequeue() == item

    assert queue.is_empty() == True
    for i in range(10):
        for item in items:
            queue.enqueue(item)
    for i in range(10):
        for i in range(len(items)):
            assert queue.dequeue() == items[i % len(items)]
        print(queue)
    

    print(queue)
    

test_array_queue_4([1, 2, 3, 4, 5])
test_array_queue_4([])
        

[1, 2, 3, 4, 5]
[None, 2, 3, 4, 5]
[None, None, 3, 4, 5]
[None, None, None, 4, 5]
[None, None, None, None, 5]
[None, None, None, None, None, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1,

In [12]:
class Deque:
    def add_first(self, e):
        pass
    def add_last(self, e):
        pass
    def remove_first(self):
        pass
    def remove_last(self, e):
        pass
    def first(self):
        pass
    def last(self):
        pass
    def resize(self, capacity):
        pass
    def is_empty(self):
        pass
    def _check_empty(self):
        if self.is_empty():
            raise Empty("Deque is empty")
    def __len__(self):
        pass
    

In [23]:
class ArrayDeque1(Deque):
    """circular resizing array deque"""
    MINIMUM_CAPACITY = 8
    def __init__(self, capacity):
        self._items = [None] * capacity
        self._size = 0
        self._front = 0
        self._capacity = capacity
    def __front_mm(self):
        self._front = (self._front - 1) % self._capacity
    def __front_pp(self):
        self._front = (self._front + 1) % self._capacity
    def __size_mm(self):
        self._size-=1
    def __size_pp(self):
        self._size+=1
    def __len__(self):
        return self._size
    def is_empty(self):
        return len(self) == 0
    def first(self):
        self._check_empty()
        return self._items[self._front]
    def last(self):
        self._check_empty()
        return self._items[(self._front + self._size - 1) % self._capacity]
    def _resize(self, capacity):
        old = self._items
        new = [None] * capacity
        walk = self._front
        for k in range(self._size):
            new[k] = old[walk]
            walk = (1 + walk) % self._capacity
        self._front = 0
        self._capacity = capacity
        self._items = new
    def add_first(self, e):
        if self._size == self._capacity:
            self._resize(2 * self._capacity)
        # self._front = (self._front - 1) % self._capacity
        self.__front_mm()
        self._items[self._front] = e
        self.__size_pp()
    def add_last(self, e):
        if self._size == self._capacity:
            self._resize(2 * self._capacity)
        avail = (self._front + self._size) % self._capacity
        self._items[avail] = e
        self.__size_pp()
    def remove_first(self):
        self._check_empty()
        item = self._items[self._front]
        self._items[self._front] = None
        # self._front = (self._front + 1) % self._capacity
        self.__front_pp()
        self.__size_mm()
        if 0 < self._size and self._size < self._capacity // 4 and self._capacity // 2 > ArrayDeque1.MINIMUM_CAPACITY:
            self._resize(self._capacity // 2)
        return item
    def remove_last(self):
        self._check_empty()
        item = self._items[(self._front + self._size - 1) % self._capacity]
        self._items[(self._front + self._size - 1) % self._capacity] = None
        self.__size_mm()
        if 0 < self._size and self._size < self._capacity // 4 and self._capacity // 2 > ArrayDeque1.MINIMUM_CAPACITY:
            self._resize(self._capacity // 2)
        return item
    def __str__(self):
        return str(self._items)

In [26]:
def test_array_deque1(items):
    deque = ArrayDeque1(len(items))
    for item in items:
        deque.add_first(item)
    if not deque.is_empty():
        deque.first() == items[0]
        assert deque.is_empty() == False
    print(deque)
    for i in range(len(items)):
        if i%2 == 0:
            deque.remove_first()
        else:
            deque.remove_last()
        print(deque)

    assert deque.is_empty() == True
    for i in range(5):
        for item in items:
            deque.add_first(item)
    print(deque)
    for i in range(5):
        for item in items:
            deque.add_last(item)
    print(deque)
    for i in range(10):
        for i in range(len(items)):
            deque.remove_last()
        print(deque)
    

    print(deque)
test_array_deque1([1, 2, 3, 4, 5])


[5, 4, 3, 2, 1]
[None, 4, 3, 2, 1]
[None, 4, 3, 2, None]
[None, None, 3, 2, None]
[None, None, 3, None, None]
[None, None, None, None, None]
[5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 5, 4, 3, 2, 1]
[5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[5, 4, 3, 2, 1, 

In [31]:
from collections import deque
def test_collections_deque(items):
    D = deque(items)
    if not len(D) == 0:
        D.popleft()
        assert (len(D) == 0) == False
    print(D)
    try:
        for i in range(len(items)):
            if i%2 == 0:
                D.popleft()
            else:
                D.pop()
            print(D)
    except IndexError as indexError:
        print('error: ', indexError)
    assert (len(D) == 0) == True
    for i in range(5):
        for item in items:
            D.appendleft(item)
    print(D)
    for i in range(5):
        for item in items:
            D.append(item)
    print(D)
    for i in range(10):
        for i in range(len(items)):
            D.pop()
        print(D)
    

    print(D)

test_collections_deque([1, 2, 3, 4, 5])

deque([2, 3, 4, 5])
deque([3, 4, 5])
deque([3, 4])
deque([4])
deque([])
error:  pop from an empty deque
deque([5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1])
deque([5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5])
deque([5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5])
deque([5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5])
deque([5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5])
deque([5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 1, 2, 3, 4, 5])
deque([5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1])
deque([5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1, 5, 4, 3, 2, 1])
deque([5, 4, 3, 2, 1, 